<a href="https://colab.research.google.com/github/HedersonSantos/Noticias/blob/main/preProcessamento_noticias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
from google.colab import files, drive
from pathlib import Path
from sklearn.model_selection import train_test_split
import io
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth', None)

### Montando drive Google e setando variáveis

In [4]:
drive.mount('/gdrive')
%cd /gdrive/My\ Drive/Colab\ Notebooks/nlp_tcc
path = '/gdrive/My Drive/Colab Notebooks/nlp_tcc'

Mounted at /gdrive
/gdrive/My Drive/Colab Notebooks/nlp_tcc


### Upload da base de noticias

In [5]:
#csvNoticias = files.upload()  
!rm *.*
!wget https://raw.githubusercontent.com/HedersonSantos/Noticias/main/noticia_tratada.zip
!unzip noticia_tratada.zip
%ls



--2021-06-04 19:30:24--  https://raw.githubusercontent.com/HedersonSantos/Noticias/main/noticia_tratada.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6432391 (6.1M) [application/zip]
Saving to: ‘noticia_tratada.zip’

noticia_tratada.zip 100%[===================>]   6.13M  31.6MB/s    in 0.2s    

2021-06-04 19:30:24 (31.6 MB/s) - ‘noticia_tratada.zip’ saved [6432391/6432391]

Archive:  noticia_tratada.zip
  inflating: noticia_tratada.csv     
noticia_tratada.csv  noticia_tratada.zip


In [6]:
#dfNews_all = pd.read_csv(io.BytesIO(csvNoticias['noticia_tratada.csv']), header=0, delimiter=';')
dfNews_all = pd.read_csv('noticia_tratada.csv', header=0, delimiter=';')
print(dfNews_all.columns)
dfNews_all.shape

Index(['FONTE', 'CATEGORIA', 'TEXTO', 'URL', 'PUBLISHEDAT', 'TITLE', 'AUTHOR',
       'DT_REGISTRO', 'TEXT_TRATADO'],
      dtype='object')


(3994, 9)

### Tratamento da CATEGORIA

In [7]:
print('Dimensao Inicial:',dfNews_all.shape)
#Elimina noticias duplicadas
dfNews = dfNews_all.drop_duplicates(subset=['URL'],keep='first').copy()
#Elimina noticias sem texto
dfNews = dfNews[(~dfNews['TEXT_TRATADO'].isnull())]
print('Dimensão após eliminar duplicadas e textos em branco:',dfNews.shape)
#Tira as marcações do campo Text[] do Postgres em categoria e seta para minúscula
dfNews['CATEGORIA'] = dfNews.loc[:,['CATEGORIA']].apply(lambda x: x['CATEGORIA'].replace('[','').replace(']','').replace('"','').replace(", 'redacao'",'').replace("'",''),axis=1)
dfNews['CATEGORIA'] = dfNews.loc[:,['CATEGORIA']].apply(lambda x: x['CATEGORIA'].lower(),axis=1)
#Agrupa as notícias locais como categoria LOCAL
siglaEstados = ['ac','al','ap','am','ba','ce','df','es','go','ma','mt','ms','mg','pa','pb','pr','pe','pi','rj','rn','rs','ro','rr','sc','sp','se','to']
nomeEstados = ['acre','alagoas','amapa','amazonas','bahia','ceara','distrito-federal','espirito-santo','goias','maranhao','mato-grosso','mato-grosso-sul','minas-gerais','minas',
               'para','paraiba','parana','pernambuco','piaui','rio-janeiro','rio-grande-norte','rio-grande-sul','rondonia','roraima','santa-catarina','sao-paulo','sergipe','tocantins']
nomeCidades =['rio','bh',]
local = siglaEstados + nomeEstados + nomeCidades
dfNews['CATEGORIA'] = dfNews.loc[:,['CATEGORIA']].apply(lambda x: 'local' if x['CATEGORIA'] in local else x['CATEGORIA'],axis=1)
#efe, afp, rfi são notícias internacionais. 
mundo = ['efe','afp','rfi','mundo','internacional']
dfNews['CATEGORIA'] = dfNews.loc[:,['CATEGORIA']].apply(lambda x: 'mundo' if any( i in x['CATEGORIA'] for i in mundo) else x['CATEGORIA'],axis=1)
#Na lista miscelanea há as categorias que possuem miscelâneas de noticias de outras categorias.
miscelanea = ['deutschewelle', '','bbc','ansa','universa']
dfNews['CATEGORIA'] = dfNews.loc[:,['CATEGORIA']].apply(lambda x: 'miscelanea' if x['CATEGORIA'] in miscelanea else x['CATEGORIA'],axis=1)
#'reuters' é uma categoria especifica do UOL. Possui textos de economia e miscelânea de noticias.
dfreuters = dfNews[dfNews['CATEGORIA']=='reuters'].copy()
dfreuters['CATEGORIA'] = dfreuters.loc[:,['CATEGORIA','URL']].apply(lambda x: 'economia' if 'economia' in x['URL']  else 'miscelanea',axis=1)
dfNews=dfNews[dfNews['CATEGORIA']!='reuters']
dfNews = pd.concat([dfNews,dfreuters],sort=False)
#noticias é uma categoria da globo. Traz noticias do Jornal Nacional. São miscelâneas.
dfnoticia = dfNews[dfNews['CATEGORIA']=='noticia'].copy()
dfnoticia['CATEGORIA'] = dfnoticia.loc[:,['CATEGORIA','URL']].apply(lambda x: 'saude' if 'ciencia-e-saude' in x['URL']  else 'miscelanea',axis=1)
dfNews=dfNews[dfNews['CATEGORIA']!='noticia']
dfNews = pd.concat([dfNews,dfnoticia],sort=False)
#redacao é uma categoria da uol. Traz noticias do meio-ambiente. VERIFICAR QUANDO ATUALIZAR O DATASET SE PERMANECE ASSIM.
dfNews['CATEGORIA'] = dfNews.loc[:,['CATEGORIA','URL']].apply(lambda x: 'meio-ambiente' if ('meio-ambiente' in x['URL']) and
                                                              (x['CATEGORIA']=='redacao') else x['CATEGORIA'],axis=1)
dfNews['CATEGORIA'] = dfNews.loc[:,['CATEGORIA','URL']].apply(lambda x: 'meio-ambiente' if ('meio-ambiente' in x['URL']) and
                                                              (x['CATEGORIA']=='sociedade') else 
                                                              'miscelanea' if x['CATEGORIA']=='sociedade' else x['CATEGORIA'] ,axis=1)
dfNews['CATEGORIA'] = dfNews.loc[:,['CATEGORIA']].apply(lambda x: 'saude' if any(i in x['CATEGORIA'] for i in ['vivabem','bemestar'])  else x['CATEGORIA'] ,axis=1)
dfNews['CATEGORIA'] = dfNews.loc[:,['CATEGORIA']].apply(lambda x: 'esporte' if any(i in x['CATEGORIA'] for i in ['futebol','volei']) else x['CATEGORIA'],axis=1)
midia = ['famosos','cinema','oscar','videos','bbb','musica','lives','cultura']
dfNews['CATEGORIA'] = dfNews.loc[:,['CATEGORIA']].apply(lambda x: 'midia' if any(i in x['CATEGORIA'] for i in midia) else x['CATEGORIA'],axis=1)
dfNews['CATEGORIA'] = dfNews.loc[:,['CATEGORIA']].apply(lambda x: 'politica' if any(i in x['CATEGORIA'] for i in ['politica'])  else x['CATEGORIA'],axis=1)
dfNews.shape

Dimensao Inicial: (3994, 9)
Dimensão após eliminar duplicadas e textos em branco: (3991, 9)


(3991, 9)

In [8]:
#Filtra categorias com poucas noticias. 
dfNewsGrupo = dfNews.groupby(['CATEGORIA'])['URL'].count().reset_index().sort_values('URL',ascending=False)
dfNewsGrupo = dfNewsGrupo[dfNewsGrupo['URL']<=30]
dfNews = dfNews[~dfNews['CATEGORIA'].isin(dfNewsGrupo['CATEGORIA'].tolist())]
dfNews.shape

(3837, 9)

In [47]:
#Criando coluna CATEGORIA numerada.
dfNewsGrupo = dfNews.groupby(['CATEGORIA'])['URL'].count().reset_index().sort_values('URL',ascending=False)
print(dfNewsGrupo)
dictCategoriaNumero = dict(zip(dfNewsGrupo['CATEGORIA'],range(1,30)))
print(dictCategoriaNumero)
dfNews['CAT_NRO'] = dfNews['CATEGORIA'].map(dictCategoriaNumero)
dfNews[['CATEGORIA','CAT_NRO']].head(5)

        CATEGORIA  URL
7           local  673
12       politica  622
11          mundo  528
10     miscelanea  350
13          saude  303
5         esporte  253
4        economia  219
9           midia  219
0          brasil  189
2     coronavirus  172
1         colunas  128
6         justica   76
8   meio-ambiente   41
3       cotidiano   33
14     tecnologia   31
{'local': 1, 'politica': 2, 'mundo': 3, 'miscelanea': 4, 'saude': 5, 'esporte': 6, 'economia': 7, 'midia': 8, 'brasil': 9, 'coronavirus': 10, 'colunas': 11, 'justica': 12, 'meio-ambiente': 13, 'cotidiano': 14, 'tecnologia': 15}


,CATEGORIA,CAT_NRO
0,esporte,6
1,colunas,11
2,miscelanea,4
3,miscelanea,4
4,miscelanea,4


In [39]:
[v  for v in range(1,20) if v%2==0 ]

[2, 4, 6, 8, 10, 12, 14, 16, 18]

### Prepara DataSet Treinamento, Teste

In [10]:
dfMiscelanea = dfNews[dfNews['CATEGORIA']=='miscelanea'].loc[:,['TEXT_TRATADO']]
print('miscelanea:',dfMiscelanea.shape)



miscelanea: (350, 1)


In [24]:
dfNews.columns

Index(['FONTE', 'CATEGORIA', 'TEXTO', 'URL', 'PUBLISHEDAT', 'TITLE', 'AUTHOR',
       'DT_REGISTRO', 'TEXT_TRATADO'],
      dtype='object')

In [27]:
#vamos obter uma amostra randômica de 130 noticias para cada CATEGORIA e dividir em dois datasets Treinamento e Teste.
dfNewsGrupo = dfNewsGrupo[dfNewsGrupo['CATEGORIA']!='miscelanea']
n=180
categorias = dfNewsGrupo[dfNewsGrupo['URL']>n].loc[:,['CATEGORIA']]['CATEGORIA'].to_list()
print(categorias)
frame=[]
for categoria in categorias:
  dftmp = dfNews[dfNews['CATEGORIA']==categoria].loc[:,['TEXTO','CATEGORIA']].sample(n=n)
  frame.append(dftmp)
dfAmostra = pd.concat(frame,ignore_index=True, sort=False)
#criar coluna categoria para inteiro
i=0
for cat in categorias:
  dfAmostra = dfAmostra.replace(cat,i)
  i+=1
print('Amostra:', dfAmostra.shape)

df_train, df_test = train_test_split(dfAmostra, train_size = 0.9, random_state = 10)
print('Treinamento:',df_train.shape)
print('Teste:', df_test.shape)



['local', 'politica', 'mundo', 'saude', 'esporte', 'economia', 'midia', 'brasil']
Amostra: (1440, 2)
Treinamento: (1296, 2)
Teste: (144, 2)


In [25]:
categoria = dfAmostra['CATEGORIA'].unique()
v = dict(zip(categoria, range(1,20)))
dfAmostra.replace(v)

In [28]:
dfAmostra['CATEGORIA']

0          local
1          local
2          local
3          local
4          local
5          local
6          local
7          local
8          local
9          local
10         local
11         local
12         local
13         local
14         local
15         local
16         local
17         local
18         local
19         local
20         local
21         local
22         local
23         local
24         local
25         local
26         local
27         local
28         local
29         local
30         local
31         local
32         local
33         local
34         local
35         local
36         local
37         local
38         local
39         local
40         local
41         local
42         local
43         local
44         local
45         local
46         local
47         local
48         local
49         local
50         local
51         local
52         local
53         local
54         local
55         local
56         local
57         local
58         loc

In [45]:
df_train.to_csv(path + "/train.csv",header=True,doublequote=True, index=False, quotechar='"', encoding='utf-8')
df_test.to_csv(path + "/test.csv",header=True,doublequote=True, index=False, quotechar='"', encoding='utf-8')


In [43]:
dft.head(4)

TEXTO  \
475                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  Por G1 \n Por G1 \n  17/04/2021 11h39    Atualizado  17/04/2021    \n Rainha Elizabeth II, casada por mais de 70 anos com Philip, é a última a seguir no cortejo durante o funeral de 17 de abril de 2021 — Foto: Leon Neal/Pool/Reuters \n O funeral do príncipe Philip, que morreu aos 99 anos, aconteceu neste sábado (17) em uma cerimônia reduzida por conta dos protocolos da Covid-19. Despedida foi acompanhada pela rainha Elizabeth II e netos, inclusive o príncipe Harry. \n O corpo do duque de Edimburgo foi levado para a cripta real. Antes, o corpo do príncipe Philip foi velado na Capela de São Jorge, dentro da propriedade real do Castelo de Windsor, onde ele morava com a rainha Elizabeth II. \n Príncipes William e Harry acompanham funeral do avô, príncipe Philip — Foto: Jonathan Brady/Reuters \n Após o fim da cerimônia, a família real retornou ao Castelo de Windsor. \nRestrições Covid\n Por conta das restrições impostas durante a pandemi

In [37]:
dft = pd.read_csv('train.csv')
dft[dft['CATEGORIA']=='midia'].head(5)

TEXTO  \
1247  Colaboração para o UOL\nColaboração para o UOL\n28/04/2021 01h05 \nO capítulo de hoje de Chiquititas, novela do SBT escrita por Iris Abravanel, tem como destaque aproximação de Miguel, e vai ao ar às 20h50.\nLeia abaixo o resumo de hoje:\nCarol conta que eles arcam com todas as despesas, porém terá que ir sozinha e precisará morar lá por um ano. Mili está aflita por estar sem inspiração para escrever. Miguel aproveita que Mili dorme no pátio e deixa uma rosa branca ao lado dela com um bilhete dizendo que a flor é para ajudá-la a voltar com a inspiração. Carmen manda Armando continuar com a farsa para deixar as vagas lembranças de Gabriela ainda mais confusas. Gabriela está sem memória e pensa se chamar Sofia. Os meninos começam a treinar para o torneio de Artes Marciais. Maria Cecília se incomoda com o excesso de bagunça que seu recém marido, Tobias, anda deixando no apartamento.\nTati não aceita bem a notícia do trabalho de modelo de Vivi e fica triste. Tati chora por pensar que Vivi prefere ser modelo do que sua irmã. Vivi pede conselho para Samuca, pois está em dúvida se aceita ou não, a proposta para modelar por um ano no Rio Grande do Sul. Cris mostra para Samuca como está ficando sua matéria sobre namoro. Carol e Junior combinam que a partir de agora vão sempre conversar sobre qualquer achismo ou mal entendido. Vivi está muito emocionada e conta para Tati que decidiu aceitar a proposta e que não pode desistir de seus sonhos. A garota conta que se pudesse levaria Tati, mas que essa pode ser uma oportunidade única.\nMili tenta descobrir quem foi que deixou uma rosa branca na cama dela enquanto dormia. Tati também se emociona e chama a irmã de egoísta. Chico conversa com Vivi, e Mili com Tati. Beto e Érica saem para um encontro num bar. Em casa, Carol prepara um jantar com a ajuda de Junior. Os dois se beijam enquanto preparam uma massa. Mili conta uma nova história para as meninas antes de dormir. A história é sobre dois passarinhos, inspirada no conflito entre Tati e Vivi.\n Carol conta que eles arcam com todas as despesas, porém terá que ir sozinha e precisará morar lá por um ano. Mili está aflita por estar sem inspiração para escrever. Miguel aproveita que Mili dorme no pátio e deixa uma rosa branca ao lado dela com um bilhete dizendo que a flor é para ajudá-la a voltar com a inspiração. Carmen manda Armando continuar com a farsa para deixar as vagas lembranças de Gabriela ainda mais confusas. Gabriela está sem memória e pensa se chamar Sofia. Os meninos começam a treinar para o torneio de Artes Marciais. Maria Cecília se incomoda com o excesso de bagunça que seu recém marido, Tobias, anda deixando no apartamento. Tati não aceita bem a notícia do trabalho de modelo de Vivi e fica triste. Tati chora por pensar que Vivi prefere ser modelo do que sua irmã. Vivi pede conselho para Samuca, pois está em dúvida se aceita ou não, a proposta para modelar por um ano no Rio Grande do Sul. Cris mostra para Samuca como está ficando sua matéria sobre namoro. Carol e Junior combinam que a partir de agora vão sempre conversar sobre qualquer achismo ou mal entendido. Vivi está muito emocionada e conta para Tati que decidiu aceitar a proposta e que não pode desistir de seus sonhos. A garota conta que se pudesse levaria Tati, mas que essa pode ser uma oportunidade única. Mili tenta descobrir quem foi que deixou uma rosa branca na cama dela enquanto dormia. Tati também se emociona e chama a irmã de egoísta. Chico conversa com Vivi, e Mili com Tati. Beto e Érica saem para um encontro num bar. Em casa, Carol prepara um jantar com a ajuda de Junior. Os dois se beijam enquanto preparam uma massa. Mili conta uma nova história para as meninas antes de dormir. A história é sobre dois passarinhos, inspirada no conflito entre Tati e Vivi. \n Carol conta que eles arcam com todas as despesas, porém terá que ir sozinha e precisará morar lá por um ano. Mili está aflita por estar sem inspiração para escrever. Miguel aproveita que Mili dor